In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
model_data = pd.read_parquet('../data/model_data.parquet')

In [ ]:
model_data.drop(columns=['Patient ID', 'Follow-up #'], inplace=True)

Now it's time for normalization. We want to do on the train dataset, after splitting data - to prevent data leakage, we don't want the information from val/test set in our model during training

In [ ]:

normalization_params = {"min_age": model_data['Patient Age'].min(),
                        "max_age": model_data['Patient Age'].max()}

model_data['PatientAge_norm'] = (model_data['Patient Age'] - model_data['Patient Age'].min()) / (model_data['Patient Age'].max() - model_data['Patient Age'].min())
model_data.drop(columns=['Patient Age'], inplace=True)
